In [25]:
import torch
import torchvision
import cv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [26]:
import os
import cv2
import pandas as pd

os.chdir('D:\\Handwriting-Recognition-Project')
print(os.getcwd())

datasetPATH = os.path.join('Datasets', 'archive', 'iam_words')
wordsPATH = os.path.join(datasetPATH, 'words')
labelsPATH = os.path.join('Datasets', 'archive', 'words_new.txt')

print("Dataset Path:", datasetPATH)
print("Words Path:", wordsPATH)
print("Labels Path:", labelsPATH)


D:\Handwriting-Recognition-Project
Dataset Path: Dataset\archive\iam_words
Words Path: Dataset\archive\iam_words\words
Labels Path: Dataset\archive\words_new.txt


In [31]:
def loadLabels(labelsFile = labelsPATH, wordsPATH = wordsPATH):
    labels = []

    with open(labelsFile, 'r') as file:
        for line in file:
            if line.startswith('#'):  #Ignore the commented lines
                continue

            lineParts = line.strip().split()
            if len(lineParts) < 8:
                print(f"Skipping line due to insufficient fields: '{line.strip()}'")
                continue

            wordID, segmentationStatus = lineParts[0], lineParts[1]

            try:
                grayLevel = int(lineParts[2]) if len(lineParts) > 2 else None
                x = int(lineParts[3]) if len(lineParts) > 3 else None
                y = int(lineParts[4]) if len(lineParts) > 4 else None
                w = int(lineParts[5]) if len(lineParts) > 5 else None
                h = int(lineParts[6]) if len(lineParts) > 6 else None
                grammaticalTag = lineParts[7] if len(lineParts) > 7 else None
                
                word = " ".join(lineParts[8:]) if len(lineParts) > 8 else None

                folder = wordID.split('-')[0]
                subFolder = folder + '-' + wordID.split('-')[1]
                image = f"{wordID}.png"
                imagePath = os.path.join(wordsPATH, folder, subFolder, image)

                if os.path.exists(imagePath):
                    labels.append({
                        'wordID': wordID,
                        'segmentationStatus': segmentationStatus,
                        'grayLevel': grayLevel,
                        'boundingBoxX': x,
                        'boundingBoxY': y,
                        'boundingBoxWidth': w,
                        'boundingBoxHeight': h,
                        'grammaticalTag': grammaticalTag,
                        'word': word,
                        'imagePath': imagePath
                    })
                else:
                    print(f"Image not found for imagePath {imagePath}, skipping.")
            
            except ValueError as e:
                print(f"Skipping line due to parsing error in integer fields: '{line.strip()}' - {e}")
                
    return pd.DataFrame(labels)


In [36]:
detailedLabels = loadLabels(labelsPATH, wordsPATH)
print(detailedLabels.head())
csvPath = os.path.join('Data', 'Labels.csv')
detailedLabels.to_csv(csvPath, index=False)

           wordID segmentationStatus  grayLevel  boundingBoxX  boundingBoxY  \
0  a01-000u-00-00                 ok        154           408           768   
1  a01-000u-00-01                 ok        154           507           766   
2  a01-000u-00-02                 ok        154           796           764   
3  a01-000u-00-03                 ok        154           919           757   
4  a01-000u-00-04                 ok        154          1185           754   

   boundingBoxWidth  boundingBoxHeight grammaticalTag  word  \
0                27                 51             AT     A   
1               213                 48             NN  MOVE   
2                70                 50             TO    to   
3               166                 78             VB  stop   
4               126                 61            NPT   Mr.   

                                           imagePath  
0  Dataset\archive\iam_words\words\a01\a01-000u\a...  
1  Dataset\archive\iam_words\words\a